## 5.2 워드 임베딩

### 텍스트를 숫자로 표현

이 튜토리얼에는 단어 임베딩에 대한 소개가 포함되어 있습니다. 감정 분류 작업을 위해 간단한 Keras 모델을 사용하여 자신 만의 단어 임베딩을 훈련 한 다음 [Embedding Projector](http://projector.tensorflow.org) (아래 이미지 참조)에서 시각화합니다.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding.jpg?raw=1" alt="Screenshot of the embedding projector" width="400"/>


기계 학습 모델은 벡터 (숫자 배열)를 입력으로 사용합니다. 텍스트로 작업 할 때 가장 먼저해야 할 일은 문자열을 모델에 공급하기 전에 문자열을 숫자로 변환 (또는 텍스트를 "벡터화")하는 전략을 마련하는 것입니다. 이 섹션에서는이를위한 세 가지 전략을 살펴 봅니다.

### 원-핫 인코딩

첫 번째 아이디어로 어휘의 각 단어를 "원-핫"인코딩 할 수 있습니다. "고양이가 매트 위에 앉았다"라는 문장을 생각해보십시오. 이 문장의 어휘 (또는 고유 한 단어)는 (cat, mat, on, sat, the)입니다. 각 단어를 나타 내기 위해 어휘와 길이가 같은 0 벡터를 만든 다음 해당 단어에 해당하는 색인에 1을 배치합니다. 이 접근 방식은 다음 다이어그램에 나와 있습니다.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/one-hot.png?raw=1" alt="Diagram of one-hot encodings" width="400" />

문장의 인코딩을 포함하는 벡터를 만들기 위해 각 단어에 대해 원-핫 벡터를 연결할 수 있습니다.

요점 : 이 접근 방식은 비효율적입니다. 원-핫 인코딩 된 벡터는 희소합니다 (즉, 대부분의 인덱스가 0 임). 어휘에 10,000 개의 단어가 있다고 상상해보십시오. 각 단어를 원-핫 인코딩하려면 요소의 99.99 %가 0 인 벡터를 만듭니다.

### 각 단어를 고유 한 번호로 인코딩

두 번째 방법은 고유 한 숫자를 사용하여 각 단어를 인코딩하는 것입니다. 위의 예를 계속하여 "cat"에 1을 할당하고 "mat"에 2를 할당 할 수 있습니다. 그런 다음 "The cat sat on the mat"라는 문장을 [5, 1, 4, 3, 5, 2]와 같은 고밀도 벡터로 인코딩 할 수 있습니다. 이 접근 방식은 효율적입니다. 희소 벡터 대신 이제 모든 요소가 가득 찬 조밀 한 벡터가 있습니다.

그러나이 방법에는 두 가지 단점이 있습니다.

* 정수 인코딩은 임의적입니다 (단어 간의 관계를 캡처하지 않음).

* 정수 인코딩은 모델이 해석하기 어려울 수 있습니다. 예를 들어 선형 분류기는 각 특성에 대한 단일 가중치를 학습합니다. 두 단어의 유사성과 해당 인코딩의 유사성 간에는 관계가 없기 때문에이 기능 가중치 조합은 의미가 없습니다.

### 단어 임베딩

단어 임베딩은 유사한 단어가 유사한 인코딩을 갖는 효율적이고 조밀 한 표현을 사용하는 방법을 제공합니다. 중요한 것은이 인코딩을 직접 지정할 필요가 없다는 것입니다. 임베딩은 부동 소수점 값으로 구성된 조밀 한 벡터입니다 (벡터의 길이는 사용자가 지정하는 매개 변수 임). 임베딩 값을 수동으로 지정하는 대신 학습 가능한 매개 변수입니다 (모델이 조밀 한 레이어에 대한 가중치를 학습하는 것과 같은 방식으로 학습 중에 모델에서 학습 한 가중치). 8 차원 (작은 데이터 세트의 경우), 대규모 데이터 세트로 작업 할 때 최대 1024 차원의 단어 임베딩을 보는 것이 일반적입니다. 더 높은 차원의 임베딩은 단어 간의 세밀한 관계를 캡처 할 수 있지만 학습하는 데 더 많은 데이터가 필요합니다.

<img src="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/text/images/embedding2.png?raw=1" alt="Diagram of an embedding" width="400"/>

위는 단어 임베딩에 대한 다이어그램입니다. 각 단어는 부동 소수점 값의 4 차원 벡터로 표시됩니다. 임베딩을 생각하는 또 다른 방법은 "조회 테이블"입니다. 이러한 가중치를 학습 한 후 표에서 해당하는 조밀 한 벡터를 찾아 각 단어를 인코딩 할 수 있습니다.

In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from datetime import datetime
from tensorflow.keras import Model, Sequential
from tensorflow.keras.layers import Activation, Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

### IMDb 데이터 세트 다운로드

튜토리얼을 통해 Large Movie Review Dataset 을 사용하게됩니다. 이 데이터 세트에서 감정 분류기 모델을 훈련시키고 그 과정에서 임베딩을 처음부터 배우게됩니다. 데이터 세트를 처음부터로드하는 방법에 대한 자세한 내용은 텍스트로드 가이드를 참조 하세요 .

Keras 파일 유틸리티를 사용하여 데이터 세트를 다운로드하고 디렉토리를 살펴보십시오.

In [2]:
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                    untar=True, cache_dir='.',
                                    cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

84131840/84125825 [==============================] - 447s 5us/step


['imdb.vocab', 'imdbEr.txt', 'README', 'test', 'train']

train/ 디렉토리를 살펴보십시오. 그것은 각각 긍정적 및 부정으로 표시된 영화 리뷰가있는 pos 및 neg 폴더를 가지고 있습니다. 이진 분류 모델을 학습하기 위해 pos 및 neg 폴더의 리뷰를 사용합니다.

In [3]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'neg',
 'pos',
 'unsup',
 'unsupBow.feat',
 'urls_neg.txt',
 'urls_pos.txt',
 'urls_unsup.txt']

train 디렉터리에는 교육 데이터 세트를 만들기 전에 제거해야하는 추가 폴더도 있습니다.

In [4]:
remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

다음으로tf.data.Dataset 사용하여 tf.keras.preprocessing.text_dataset_from_directory . 이 텍스트 분류 자습서 에서이 유틸리티 사용에 대한 자세한 내용을 읽을 수 있습니다.

train 디렉터리를 사용하여 유효성 검사를 위해 20 %로 분할 된 기차 및 유효성 검사 데이터 세트를 모두 만듭니다.

In [5]:
batch_size = 1024
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2, 
    subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2, 
    subset='validation', seed=seed)

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


훈련 데이터 세트에서 몇 가지 영화 리뷰와 라벨 (1: positive, 0: negative) 을 살펴보세요.


In [6]:
for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(label_batch[i].numpy(), text_batch.numpy()[i])

0 b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"
1 b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This movie is a tad bit "grease"esk (without

### 성능을위한 데이터 세트 구성

이는 I / O가 차단되지 않도록 데이터를로드 할 때 사용해야하는 두 가지 중요한 방법입니다.

.cache() 데이터가 디스크에서로드 된 후 메모리에 데이터를 보관합니다. 이렇게하면 모델을 학습하는 동안 데이터 세트가 병목 현상이 발생하지 않습니다. 데이터 세트가 너무 커서 메모리에 맞지 않는 경우이 방법을 사용하여 성능이 뛰어난 온 디스크 캐시를 생성 할 수도 있습니다. 이는 많은 작은 파일보다 읽기가 더 효율적입니다.

.prefetch() 학습 중에 데이터 전처리 및 모델 실행과 겹칩니다.

In [22]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

### Embedding 레이어 사용

Keras를 사용하면 단어 임베딩을 쉽게 사용할 수 있습니다. Embedding 레이어를 살펴보십시오.

임베딩 레이어는 정수 인덱스 (특정 단어를 의미)에서 고밀도 벡터 (임베딩)로 매핑하는 조회 테이블로 이해할 수 있습니다. 임베딩의 차원 (또는 너비)은 Dense 레이어의 뉴런 수를 실험하는 것과 같은 방식으로 문제에 잘 맞는지 확인하기 위해 실험 할 수있는 매개 변수입니다.


In [23]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)

Embedding 레이어를 만들면 다른 레이어와 마찬가지로 임베딩에 대한 가중치가 임의로 초기화됩니다. 훈련 중에 역 전파를 통해 점진적으로 조정됩니다. 학습 된 단어 임베딩은 모델이 학습 된 특정 문제에 대해 학습 된 것처럼 단어 간의 유사성을 대략적으로 인코딩합니다.

임베딩 레이어에 정수를 전달하면 결과는 각 정수를 임베딩 테이블의 벡터로 대체합니다.

In [24]:
result = embedding_layer(tf.constant([1,2,3]))
result.numpy()

array([[-0.03925378,  0.03625094, -0.04450548, -0.02478501,  0.02606234],
       [ 0.04069226, -0.02290428, -0.04640144,  0.03801124, -0.0055039 ],
       [ 0.04059828,  0.02623153, -0.03168575,  0.03137945,  0.03904082]],
      dtype=float32)

텍스트 또는 시퀀스 문제의 경우 Embedding 레이어는 모양 (samples, sequence_length) 의 정수 2D 텐서를 사용하며, 여기서 각 항목은 정수 시퀀스입니다. 가변 길이의 시퀀스를 포함 할 수 있습니다. 모양 (32, 10) (길이 10의 32 시퀀스 배치) 또는 (64, 15) (길이 15의 64 시퀀스 배치 (64, 15) 배치 위의 임베딩 레이어로 공급할 수 있습니다.

반환 된 텐서는 입력보다 축이 하나 더 많으며 임베딩 벡터는 새 마지막 축을 따라 정렬됩니다. (2, 3) 입력 배치를 전달하면 출력은 (2, 3, N)


In [25]:
result = embedding_layer(tf.constant([[0,1,2],[3,4,5]]))
result.shape

TensorShape([2, 3, 5])

시퀀스 배치가 입력으로 주어지면 임베딩 레이어는 (samples, sequence_length, embedding_dimensionality) 모양의 3D 부동 소수점 텐서를 반환합니다. 이 가변 길이 시퀀스에서 고정 표현으로 변환하려면 다양한 표준 접근 방식이 있습니다. Dense 레이어로 전달하기 전에 RNN, Attention 또는 풀링 레이어를 사용할 수 있습니다. 이 튜토리얼은 가장 간단하기 때문에 풀링을 사용합니다. RNN을 사용한 텍스트 분류 튜토리얼은 좋은 다음 단계입니다.

### 텍스트 전처리

다음으로 감정 분류 모델에 필요한 데이터 세트 전처리 단계를 정의합니다. 원하는 매개 변수로 TextVectorization 레이어를 초기화하여 영화 리뷰를 벡터화합니다. 텍스트 분류 튜토리얼에서이 레이어 사용에 대해 자세히 알아볼 수 있습니다.

In [27]:
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

vocab_size = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

### 분류 모델 만들기

Keras Sequential API 를 사용하여 감정 분류 모델을 정의합니다. 이 경우 "연속 단어 모음"스타일 모델입니다.

- TextVectorization 레이어는 문자열을 어휘 색인으로 변환합니다. 이미 vectorize_layer 를 TextVectorization 레이어로 초기화했으며 text_ds adapt 를 호출 adapt 어휘를 구축했습니다. 이제 vectorize_layer를 종단 간 분류 모델의 첫 번째 계층으로 사용하여 변형 된 문자열을 Embedding 계층에 공급할 수 있습니다.

- Embedding 레이어는 정수로 인코딩 된 어휘를 가져와 각 단어 인덱스에 대한 임베딩 벡터를 찾습니다. 이러한 벡터는 모델 학습으로 학습됩니다. 벡터는 출력 배열에 차원을 추가합니다. 결과 차원은 다음과 같습니다. (batch, sequence, embedding) .

- GlobalAveragePooling1D 계층은 시퀀스 차원을 평균화하여 각 예제에 대해 고정 길이 출력 벡터를 반환합니다. 이를 통해 모델은 가능한 가장 간단한 방법으로 가변 길이 입력을 처리 할 수 ​​있습니다.

- 고정 길이 출력 벡터는 16 개의 은닉 유닛이있는 완전 연결 ( Dense ) 계층을 통해 파이프됩니다.

- 마지막 레이어는 단일 출력 노드와 조밀하게 연결됩니다.

주의 : 이 모델은 마스킹을 사용하지 않으므로 제로 패딩이 입력의 일부로 사용되므로 패딩 길이가 출력에 영향을 미칠 수 있습니다. 이 문제를 해결하려면 마스킹 및 패딩 가이드를 참조하세요.

In [28]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

### 모델 컴파일 및 훈련

TensorBoard 를 사용하여 손실 및 정확도를 포함한 메트릭을 시각화합니다. tf.keras.callbacks.TensorBoard 만듭니다.

In [29]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

Adam 최적화 프로그램과 BinaryCrossentropy 손실을 사용하여 모델을 컴파일하고 훈련합니다.

In [30]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [31]:
model.fit(
    train_ds,
    validation_data=val_ds, 
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
20/20 [==============================] - 3s 130ms/step - loss: 0.6929 - accuracy: 0.4987 - val_loss: 0.6917 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 2s 79ms/step - loss: 0.6910 - accuracy: 0.4987 - val_loss: 0.6880 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 2s 78ms/step - loss: 0.6867 - accuracy: 0.4987 - val_loss: 0.6813 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 2s 78ms/step - loss: 0.6790 - accuracy: 0.4987 - val_loss: 0.6703 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 2s 79ms/step - loss: 0.6669 - accuracy: 0.4988 - val_loss: 0.6558 - val_accuracy: 0.4894
Epoch 6/15
20/20 [==============================] - 2s 78ms/step - loss: 0.6505 - accuracy: 0.5077 - val_loss: 0.6377 - val_accuracy: 0.5340
Epoch 7/15
20/20 [==============================] - 2s 81ms/step - loss: 0.6299 - accuracy: 0.5729 - val_loss: 0.6166 - val_accuracy: 0.6148
Epoch 8/15
2

이 접근 방식을 사용하면 모델이 약 84 %의 검증 정확도에 도달합니다 (학습 정확도가 더 높기 때문에 모델이 과적 합된다는 점에 유의하십시오).

참고 : 임베딩 레이어를 훈련하기 전에 가중치가 무작위로 초기화 된 방식에 따라 결과가 약간 다를 수 있습니다.

모델 요약을 살펴보고 모델의 각 계층에 대해 자세히 알아볼 수 있습니다.

In [32]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization_2 (TextVe (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d_2 ( (None, 16)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


TensorBoard에서 모델 측정 항목을 시각화합니다.

In [ ]:
%load_ext tensorboard
%tensorboard --logdir logs

![embeddings_classifier_accuracy.png](https://www.tensorflow.org/tutorials/text/images/embeddings_classifier_accuracy.png)

## 훈련 된 단어 임베딩을 검색하여 디스크에 저장

다음으로 훈련 중에 학습 한 단어 임베딩을 검색합니다. 임베딩은 모델에있는 임베딩 레이어의 가중치입니다. 가중치 행렬은 형태 (vocab_size, embedding_dimension) 입니다.

get_layer() 및 get_weights() 사용하여 모델에서 가중치를 얻습니다. get_vocabulary() 함수는 한 줄에 하나의 토큰으로 메타 데이터 파일을 빌드하기위한 어휘를 제공합니다.

In [34]:
weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

디스크에 가중치를 씁니다. Embedding Projector 를 사용하려면 벡터 파일 (임베딩 포함)과 메타 데이터 파일 (단어 포함)의 두 파일을 탭으로 구분 된 형식으로 업로드합니다.

In [36]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if  index == 0: continue # skip 0, it's padding.
  vec = weights[index] 
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [37]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception as e:
  pass